In [1]:
!pip install scikit-surprise

  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-win_amd64.whl size=723382 sha256=3d74cdb7bb4663f65f26ccfa5b20a6a4599d1b53eb492897d41832e2bcf71eb7
  Stored in directory: c:\users\hsrtc\appdata\local\pip\cache\wheels\76\44\74\b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.25,random_state=0)

In [5]:
algo = SVD()
algo.fit(trainset)

In [6]:
predictions = algo.test(testset)
print('prediction type :', type(predictions), 'size :', len(predictions))
print('prediction 결과의 최소 5개 추출')
predictions[:5]

prediction type : <class 'list'> size : 25000
prediction 결과의 최소 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.9859010049617605, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.763211361425855, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.092578252009448, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.3099498965747736, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.5595484046942065, details={'was_impossible': False})]

In [7]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]] 

[('120', '282', 3.9859010049617605),
 ('882', '291', 3.763211361425855),
 ('535', '507', 4.092578252009448)]

In [8]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.34   {'was_impossible': False}


In [9]:
accuracy.rmse(predictions)

RMSE: 0.9494


0.9494337852756208

In [14]:
import pandas as pd

ratings = pd.read_csv('documents/ratings.csv')
ratings.to_csv('documents/ratings_noh.csv', index=False, header = False)

In [17]:
from surprise import Reader 

reader = Reader(line_format = 'user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data= Dataset.load_from_file('documents/ratings_noh.csv', reader = reader)

In [18]:
trainset, testset = train_test_split(data, test_size= 0.25, random_state=0)

In [20]:
algo = SVD(n_factors=50, random_state=0)

In [21]:
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [23]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('documents/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [24]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('documents/ratings.csv')
reader = Reader(rating_scale = (0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8663  0.8763  0.8749  0.8760  0.8710  0.8729  0.0038  
MAE (testset)     0.6680  0.6715  0.6745  0.6723  0.6685  0.6709  0.0024  
Fit time          6.85    6.56    6.49    6.57    6.64    6.62    0.13    
Test time         0.21    0.18    0.35    0.26    0.18    0.23    0.06    


{'test_rmse': array([0.86633943, 0.87632347, 0.8748855 , 0.87595692, 0.87099178]),
 'test_mae': array([0.66798819, 0.67146907, 0.67448453, 0.67227935, 0.66849841]),
 'fit_time': (6.854938268661499,
  6.562483787536621,
  6.4868175983428955,
  6.5667760372161865,
  6.643340587615967),
 'test_time': (0.20839643478393555,
  0.17553067207336426,
  0.3470733165740967,
  0.26332926750183105,
  0.17653155326843262)}

In [25]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs' : [20,40,60], 'n_factors':[50,100,200]}

In [ ]:
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

In [ ]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format = 'user item rating tiemstamp', sep=',', rating_scale=(0.5,5))

data_folds = DatasetAutoFolds(ratings_file='documents/ratings_noh.csv'. reader=reader)

trainset = data_folds.build_full_trainset()

In [ ]:
algo = SVD(n_epochs= 20, n_factors=50, random_state=0)
algo.fit(trainset)